In [1]:
# import Libraries

import openai
import langchain
from pinecone import Pinecone as PineconeClient, ServerlessSpec
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAI
import os

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

In [4]:
doc=read_doc('documents/')
len(doc)

1

In [5]:
doc

[Document(page_content="I see you'd like more details about the Krutrim AI system and its prompt design. Here's a structured \noverview of how the system is calibrated:\nCore Functionality1. \nLanguage processing optimized for Indian contexts•\nReal-time response generation•\nMulti-language capabilities (including Indian regional languages)•\nPrompt Engineering Approach2. \nPre-defined response templates for common queries•\nDynamic adaptation to user questions•\nContext-aware conversation tracking•\nQuality Assurance Mechanisms3. \nAutomated fact-checking protocols•\nBias detection algorithms•\nSafety filters for sensitive topics•\nContinuous learning from verified sources•\nUser Interaction Features4. \nProgressive disclosure of information•\nClarification requests when needed•\nAppropriate level of detail based on query complexity•\nTechnical Infrastructure5. \nCloud-based processing•\nEnterprise-grade security•\nRegular system updates•\nPerformance monitoring tools•\nWould you like

In [6]:
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return doc

In [7]:
documents=chunk_data(docs=doc)
documents

[Document(page_content="I see you'd like more details about the Krutrim AI system and its prompt design. Here's a structured \noverview of how the system is calibrated:\nCore Functionality1. \nLanguage processing optimized for Indian contexts•\nReal-time response generation•\nMulti-language capabilities (including Indian regional languages)•\nPrompt Engineering Approach2. \nPre-defined response templates for common queries•\nDynamic adaptation to user questions•\nContext-aware conversation tracking•\nQuality Assurance Mechanisms3. \nAutomated fact-checking protocols•\nBias detection algorithms•\nSafety filters for sensitive topics•\nContinuous learning from verified sources•\nUser Interaction Features4. \nProgressive disclosure of information•\nClarification requests when needed•", metadata={'source': 'documents/Understanding_the_content.pdf', 'page': 0}),
 Document(page_content='Clarification requests when needed•\nAppropriate level of detail based on query complexity•\nTechnical Infr

In [8]:
len(documents)

2

In [9]:
## Embedding Technique Of OPENAI
embeddings=OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7fa74807f850>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7fa748816ac0>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [10]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

# Check if API key is loaded (without printing the actual key for security)
if os.getenv("OPENAI_API_KEY"):
    print("OpenAI API key loaded successfully")
else:
    print("Warning: OpenAI API key not found in environment variables")

OpenAI API key loaded successfully


In [11]:
vectors=embeddings.embed_query("How are you?")
vectors

[-0.016786390884603524,
 -0.012108419068213652,
 0.006708384944812083,
 -0.02596953506456214,
 -0.016144558757618204,
 0.01760102432705511,
 -0.01113332712835445,
 -0.00991754790804511,
 -0.018131772022835137,
 -0.010423608105062689,
 0.02787034767204594,
 0.0016385245805237698,
 -0.0073008457995627956,
 -0.011682587654238595,
 0.007239131243608252,
 -0.015391639405459874,
 0.028364064119682293,
 -0.011812188640838297,
 0.014033917311814754,
 -0.020588016099571123,
 0.002419215156898743,
 0.006350439961482183,
 0.0010391208324074049,
 -0.008195708490718191,
 -0.015910042420536105,
 -0.007782220360463071,
 0.025093186600494723,
 -0.012423164188338272,
 0.022340713027705957,
 -0.025179587258227856,
 0.00562837812013177,
 0.007689648293700612,
 -0.013194597301923293,
 0.004073168609241792,
 0.00882519873096743,
 -0.022340713027705957,
 0.004057739853837834,
 -0.01047915218331048,
 0.020291784740873193,
 -0.0063566113705115085,
 0.027055714229594352,
 0.0012558936118306013,
 -0.00520563264

In [12]:

len(vectors)

1536

In [13]:
## Vector Search DB In Pinecone
# Initialize Pinecone with new SDK
pc = PineconeClient(api_key=os.environ.get('PINECONE_API_KEY'))
index_name = "langchanivector"

# Create index if it doesn't exist
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,  # OpenAI embeddings dimension
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

In [14]:
index = PineconeVectorStore.from_documents(
    documents=doc,
    embedding=embeddings,
    index_name=index_name,
    pinecone_api_key=os.environ.get('PINECONE_API_KEY')
)


In [15]:
## Cosine Similarity Retreive Results from VectorDB
def retrieve_query(query,k=2):
    matching_results=index.similarity_search(query,k=k)
    return matching_results

In [16]:
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAI

In [20]:
llm=OpenAI(model="gpt-3.5-turbo-instruct",temperature=0.5)
chain=load_qa_chain(llm,chain_type="stuff")

In [21]:
## Search answers from VectorDB
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response

In [22]:
our_query = "tell me about the Quality Assurance Mechanisms"
answer = retrieve_answers(our_query)
print(answer)

[Document(page_content="I see you'd like more details about the Krutrim AI system and its prompt design. Here's a structured \noverview of how the system is calibrated:\nCore Functionality1. \nLanguage processing optimized for Indian contexts•\nReal-time response generation•\nMulti-language capabilities (including Indian regional languages)•\nPrompt Engineering Approach2. \nPre-defined response templates for common queries•\nDynamic adaptation to user questions•\nContext-aware conversation tracking•\nQuality Assurance Mechanisms3. \nAutomated fact-checking protocols•\nBias detection algorithms•\nSafety filters for sensitive topics•\nContinuous learning from verified sources•\nUser Interaction Features4. \nProgressive disclosure of information•\nClarification requests when needed•\nAppropriate level of detail based on query complexity•\nTechnical Infrastructure5. \nCloud-based processing•\nEnterprise-grade security•\nRegular system updates•\nPerformance monitoring tools•\nWould you like